In [ ]:
!pip install scikit-multilearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.4/89.4 KB 6.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from IPython.display import display
import os
import matplotlib.pyplot as plt
import math
import numpy as np
from nltk.tokenize import sent_tokenize
import string
from sklearn.model_selection import train_test_split, GridSearchCV,  RepeatedStratifiedKFold
from pandas.plotting import scatter_matrix
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator
from sklearn.pipeline import FeatureUnion
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import GaussianNB,MultinomialNB
from sklearn.svm import SVC, LinearSVC
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.metrics import accuracy_score, hamming_loss

enc=OneHotEncoder()

class DataFrameSelector(BaseEstimator):
    
    def __init__(self, attribute_names):
        self.attribute_names= attribute_names
        
    def fit(self,X, y = None):
        return self
    
    def transform(self, X):
        return X[self.attribute_names].values


local_path = '/content/'

# files = ['bahai', 'buddhism', 'christianity', 'hinduism', 'historical', 'islam','judaism', 'p_j_s', 'shinto', 't_c', 'iranian']
files = ['buddhism', 'christianity', 'hinduism', 'islam','judaism', 'p_j_s', 'shinto', 't_c']
additions = ['BU', 'CH', 'AS',  'IS', 'JU', 'AS', 'EA', 'EA',]
frames = [0]*8
results = [0]*8

def addColumn(files, additions, frames):
    for i in range(len(files)):
        csv_path=os.path.join(local_path, files[i] + '.csv')
        # results[i] = pd.read_csv(csv_path, nrows=4500)
        # print(len(results[i]))
        df = pd.read_csv(csv_path, nrows=4500) 
        df2 = df.assign(Religion=additions[i])
        frames[i] = df2

# # combine dataframes to one
addColumn(files, additions, frames)
combined_df = pd.concat(frames, ignore_index=True)
combined_df = combined_df.rename(columns={"Language of Original Document": "Language"})

# # IMPUTATION
def imputation(combined_df):
    north = ["Hong", "Macau", "Israel", "Japan", "Singapore", "Korea", "Canada", "Europe", "Russia", "Taiwan", "Australia", "Zealand", "Taiwan", "US", "UK", "Germany", "France", "Spain", "Netherlands", "Italy", "Cyprus", "Austria", "Belgium", "Bulgaria", "Croatia", "Czech", "Denmark", "Estonia", "Finland", "Greece", "Hungary", "Ireland", "Italy", "Latvia", "Lithuania", "Luxembourg", "Malta", "Poland", "Portugal", "Romania", "Slovakia", "Slovenia", "Sweden", "Iceland", "Liechtenstein", "Norway", "States", "Kingdom", "Toronto", "Switzerland"]
    # western european languages that spread as  a result of colonisation
    lang_powerhouses = ["English", "French", "Spanish", "Portuguese", "German", "Afrikaans" ]

    # year - 2000 (self feature scaling? - trying to improve results cos 0.18 before)
    #trying to drop year ...
    # combined_df['Year'] = combined_df['Year'] - 1835

#     # replace funding details with 0 if nan, 1 else
    combined_df["Funding Details"].where(combined_df["Funding Details"] != combined_df["Funding Details"], 1, inplace = True)
    combined_df["Funding Details"].where(combined_df["Funding Details"] == 1, 0, inplace = True)

#     # drop column link & column year
    combined_df.drop(columns=["Link"], axis=1, inplace=True)
    combined_df.drop(columns=["Year"], axis=1, inplace=True)


#     # not using get dummies on religion, doing one hot later
#     # replace nominal categories (RE) to numerical -> 
    combined_df = pd.get_dummies(combined_df, columns=["Religion"])

#     # remove rows where affilitations is null
    combined_df = combined_df.dropna(subset=["Affiliations"])
    combined_df.reset_index(drop=True, inplace=True)
#     print(len(combined_df))


#     # remove punctuation
    for i in range(len(combined_df)-1):
        combined_df.loc[i+1, "Affiliations"] = combined_df.loc[i+1, "Affiliations"].translate(str.maketrans('', '', string.punctuation))


#     # if affiliation in north, replace with 0
    for i in range (len(north)):
        combined_df.Affiliations = combined_df.Affiliations.apply(lambda x: '0' if north[i] in x else x)
        # try using .loc[row_indexer,col_indexer] = value instead

#     # if affiliation not in north, replace with 1
    filter  = (combined_df['Affiliations'] != '0' )
    combined_df.loc[filter, 'Affiliations'] = 1

#     # 0 value from string to int
    filter  = (combined_df['Affiliations'] == '0' )
    combined_df.loc[filter, 'Affiliations'] = 0
#     # datatype is object.. may be incorrect!


#     # replace langauge with binary tooo
#     # if lang in western powerhouses, replace with 0
    for i in range (len(lang_powerhouses)):
        combined_df.Language = combined_df.Language.apply(lambda x: '0' if lang_powerhouses[i] in x else x)

#     # if lang not in western, replace with 1
    filter  = (combined_df['Language'] != '0' )
    combined_df.loc[filter, 'Language'] = 1

#     # 0 value from string to int
    filter  = (combined_df['Language'] == '0' )
    combined_df.loc[filter, 'Language'] = 0

    return combined_df

imputed_df = imputation(combined_df)
print(imputed_df.head())
train_set, test_set = train_test_split(imputed_df, test_size= 0.3, random_state=42)
# # from now on only using train set -> df
df=train_set.copy()

df['Affiliations'] = df['Affiliations'].astype(int)
df['Language'] = df['Language'].astype(int)
df['Funding Details'] = df['Funding Details'].astype(int)

def fit():
  # fit transform uses one hot encoder for all? check..
  enc.fit(df["Affiliations"].values.reshape(-1,1))
  enc.fit(df["Language"].values.reshape(-1,1))
  enc.fit(df["Funding Details"].values.reshape(-1,1))
  # enc.fit(df["Religion"].values.reshape(-1,1))

def transform():
  enc.transform(df["Affiliations"].values.reshape(-1,1)).toarray()
  enc.transform(df["Language"].values.reshape(-1,1)).toarray()
  enc.transform(df["Funding Details"].values.reshape(-1,1)).toarray()
  # enc.transform(df["Religion"].values.reshape(-1,1)).toarray()

def fit_transform():
  fit()
  transform()

# # seperate pipelines for numerical and cetegorical data?
# # num_attribs=["Year", "Cited by"] 
num_attribs=["Cited by"] 
cat_attribs=list(df) 
# # cat_attribs.remove("Year")
cat_attribs.remove("Cited by")
print(cat_attribs)

# test here -> seemed to work indivitually, moves to fit & transform
# enc.fit(df["Religion"].values.reshape(-1,1))
# print(enc.categories_)
# enc.transform(df["Religion"].values.reshape(-1,1)).toarray()

# numeric pipeline: scales values
num_pipeline= Pipeline([
    ('selector', DataFrameSelector(num_attribs)),
    # ('imputer',SimpleImputer(strategy="median")),
    # ('attribs_adder',CombinedAttributesAdder()),
    ('std_scaler',StandardScaler())
])

# categorical pipeline: encodes 
cat_pipeline = Pipeline([
    ('selector',DataFrameSelector(cat_attribs)),
    ('one hot',OneHotEncoder())
])


full_pipeline = FeatureUnion(transformer_list=[
    ("num_pipeline",num_pipeline),
    ("cat_pipeline",cat_pipeline)
])


data_prepared = full_pipeline.fit_transform(df)
data_prepared

# set feautures and predictors after fit_transform
# religion is goal state -> goal state must be numerical? 
# data_labels=train_set["Year"].copy()
# X = df[["Year", "Cited by","Language", "Affiliations", "Funding Details"]]
X = df[["Cited by","Language", "Affiliations", "Funding Details"]]
# # y = df[['Religion_BA', 'Religion_BU', 'Religion_CR', "Religion_HD", "Religion_HI", "Religion_IS", "Religion_JU", "Religion_IN", "Religion_JA", "Religion_CH", "Religion_IR"]]
# y = df[['Religion_IR', 'Religion_BU', 'Religion_CH', 'Religion_AS', 'Religion_HI', 'Religion_IS', 'Religion_JU', 'Religion_EA']]
y = df[['Religion_BU', 'Religion_CH', 'Religion_AS', 'Religion_IS', 'Religion_JU', 'Religion_EA']]

# X
# df.Language.dtype
# df.dtypes
# # data_prepared.head()
# print(data_prepared)
# X
# X.values
# REGRESS ON YEAR - 2000

# multiclass classification - Multinomial Naive Bayes
# binary_rel_classifier = BinaryRelevance(MultinomialNB())
# binary_rel_classifier.fit(X.values,y)

# predictions = binary_rel_classifier.predict(X.values)
# print(accuracy_score(y,predictions))
# print(hamming_loss(y,predictions))

# new method - Support Vector Classification
# binary_rel_classifier = BinaryRelevance(SVC())
# binary_rel_classifier.fit(X.values,y)

# predictions = binary_rel_classifier.predict(X.values)
# print(accuracy_score(y,predictions))
# print(hamming_loss(y,predictions))

#SVC
# classifier = BinaryRelevance(
#     classifier = LinearSVC(),
#     require_dense = [False, True]
# )

# # train
# classifier.fit(X.values, y)

# # # predict
# predictions = classifier.predict(X.values)
# print(accuracy_score(y,predictions))
# print(hamming_loss(y,predictions))

# grid search
parameters = [
    {
        'classifier': [MultinomialNB()],
        'classifier__alpha': [0.7, 1.0],
    },
    # {
    #     'classifier': [SVC()],
    #     'classifier__kernel': ['rbf', 'linear'],
    # },
    {
        'classifier': [LinearSVC()],
        'require_dense': [False, True],
    },
]

# svc = SVC(random_state=0)

# cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=10, random_state=0)

# # search = GridSearchCV(estimator=svc, param_grid=param_grid, scoring="roc_auc", cv=cv)

clf = GridSearchCV(BinaryRelevance(), parameters, scoring='accuracy')
# # cv=cv throws error that data is still multilabel not multiclass
clf.fit(X.values, y)

# results_df = pd.DataFrame(clf.cv_results_)
# results_df = results_df.sort_values(by=["rank_test_score"])
# results_df = results_df.set_index(
#     results_df["params"].apply(lambda x: "_".join(str(val) for val in x.values()))
# ).rename_axis("kernel")
# results_df[["params", "rank_test_score", "mean_test_score", "std_test_score"]]

# print(clf.best_params_, clf.best_score_)



   Cited by Affiliations Funding Details Language  Religion_AS  Religion_BU  \
0         0            0               1        0            0            1   
1         0            1               0        0            0            1   
2         0            0               0        0            0            1   
3         0            0               0        0            0            1   
4        10            0               0        0            0            1   

   Religion_CH  Religion_EA  Religion_IS  Religion_JU  
0            0            0            0            0  
1            0            0            0            0  
2            0            0            0            0  
3            0            0            0            0  
4            0            0            0            0  
['Affiliations', 'Funding Details', 'Language', 'Religion_AS', 'Religion_BU', 'Religion_CH', 'Religion_EA', 'Religion_IS', 'Religion_JU']
0.0004047412546978896
0.16765924641033053
